In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [ ]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = "simdata"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [ ]:
import torch
import numpy as np
import pickle 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
import matplotlib.pyplot as plt
import re
import seaborn as sns

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian

from mlutils.measures import corr

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

In [ ]:
def get_transfer(old_experiment_name):
    # prepare the Transfer table in a way that all the info about the transferred model is in the DataFrame. Just pd.merge (on transfer_fn and transfer_hash)
    # it then with the model that the transferred model was used for. 
    transfer = pd.DataFrame(Transfer.fetch())
    transfer = pd.concat([transfer, transfer['transfer_config'].apply(pd.Series)], axis = 1).drop('transfer_config', axis = 1)

    tm = pd.DataFrame((TrainedModel * Dataset * Seed * Experiments.Restrictions & 'experiment_name = "{}"'.format(old_experiment_name)).fetch()).rename(
        columns = {'model_hash': 't_model_hash', 'trainer_hash': 't_trainer_hash', 'dataset_hash': 't_dataset_hash'})
    tm = tm.sort_values('score', ascending=False).drop_duplicates(['t_model_hash', 't_trainer_hash', 't_dataset_hash'])

    transfer = pd.merge(transfer, tm, how='inner', on=['t_model_hash', 't_trainer_hash', 't_dataset_hash'])
    transfer = pd.concat([transfer, transfer['dataset_config'].apply(pd.Series)], axis = 1).drop('dataset_config', axis = 1)
    transfer.columns = ['t_' + col if col[:2] != 't_' and col[:8] != 'transfer'  else col for col in transfer.columns]
    transfer = transfer.sort_values(['t_neuron_n', 't_image_n', 't_neuron_base_seed', 't_image_base_seed'])
    return transfer

def baseline(data, tier):
    """ Function to estimate the highest possible correlation based on the ground truth"""
    # get dataset
    for loc, row in data_.iterrows():
        dataset_config = row['dataset_config']
        dataset_config.update(seed=1)
        dataloaders = builder.get_data(row['dataset_fn'], dataset_config)
        dataset = dataloaders['train']['0-0-3-0'].dataset
        dataset.transforms = []
        break
    # Extract data
    idx = dataset.trial_info.tiers == tier
    gts = np.array([gt for gt in dataset.neurons.ground_truths]).T[idx]
    resps = np.array([datapoint.responses for datapoint in dataset])[idx]
    # Compute correlation and return
    return np.mean(corr(resps, gts, axis=0))

### Directly trained

In [ ]:
data = pd.DataFrame()
for experiment_name in ['SIM, Direct, se2d_spatialxfeaturelinear, 0-0-3', 'SIM, Direct, se2d_pointpooled, 0-0-3', 'SIM, Direct, se2d_fullgaussian2d, 0-0-3']:
    data_ = pd.DataFrame((TrainedModel * Dataset * Model * Trainer * Seed * Experiments.Restrictions & 'experiment_name="{}"'.format(experiment_name)).fetch())
    data = pd.concat([data, data_])
    
data = pd.concat([data, data['dataset_config'].apply(pd.Series)], axis = 1).drop('dataset_config', axis = 1)
data = pd.concat([data, data['model_config'].apply(pd.Series)], axis = 1).drop('model_config', axis = 1)
data['readout'] = [row.model_fn.split('.')[-1] for loc, row in data.iterrows()]
base_line = baseline(data=data, tier='validation')

In [ ]:
title = 'Directly trained'
neuron_n = 1000

hues = np.unique(data.readout.values)
fig, ax = plt.subplots(1, 1, figsize=(10,10))

sns.set_context('talk', font_scale=1.45)
pal = sns.color_palette('colorblind', n_colors=len(hues))
sns.set_palette(pal)
with sns.axes_style('ticks'):
    g = sns.pointplot('image_n', "score", hue='readout', data=data.loc[data['neuron_n'] == neuron_n], 
                hue_order=hues, ax=ax)
    plt.axhline(base_line, c='k', ls='--', label='baseline')
    g.set_ylim(0.3, 1.0)
    sns.despine(trim=True)
    g.axes.set_ylabel('validation correlation') 
    g.axes.set_xlabel('images') 
    plt.legend(loc='lower right', bbox_to_anchor=(0.99, 0.05), ncol=1, frameon=False, prop={'size': 15.})
    plt.title(title)
    
    g.figure.savefig('sim: ' + title + '.png', dpi=150)

### core-transfer (sameNI)

In [ ]:
data = pd.DataFrame()
for experiment_name in ["SIM, core_transfer (sameNI), se2d_fullgaussian2d, 0-0-3 -> 0-0-3",
                        "SIM, core_transfer (sameNI), se2d_pointpooled, 0-0-3 -> 0-0-3",
                        "SIM, core_transfer (sameNI), se2d_spatialxfeaturelinear, 0-0-3 -> 0-0-3"]:
    data_ = pd.DataFrame((TrainedModelTransfer * Dataset * Model * Trainer * Seed * Transfer.proj() * ExperimentsTransfer.Restrictions & 'experiment_name="{}"'.format(experiment_name)).fetch())
    transfer = get_transfer(old_experiment_name='SIM, Direct, {}, 0-0-3'.format(experiment_name.split(', ')[2]))
    data_ = pd.merge(data_, transfer, how='inner', on=['transfer_hash', 'transfer_fn'])
    data = pd.concat([data, data_])
data['readout'] = [row.model_fn.split('.')[-1] for loc, row in data.iterrows()]
base_line = baseline(data=data, tier='validation')

In [ ]:
title = 'Transfer: same #N, #I'
t_neuron_n = 1000

hues = np.unique(data.readout.values)
fig, ax = plt.subplots(1, 1, figsize=(10,10))

sns.set_context('talk', font_scale=1.45)
pal = sns.color_palette('colorblind', n_colors=len(hues))
sns.set_palette(pal)
with sns.axes_style('ticks'):
    g = sns.pointplot('t_image_n', "score", hue='readout', data=data.loc[data['t_neuron_n'] == t_neuron_n], 
                hue_order=hues, ax=ax)
    plt.axhline(base_line, c='k', ls='--', label='baseline')
    g.set_ylim(0.3, 1.0)
    sns.despine(trim=True)
    g.axes.set_ylabel('validation correlation') 
    g.axes.set_xlabel('images (transfer core and new readout)') 
    plt.legend(loc='lower right', bbox_to_anchor=(0.99, 0.05), ncol=1, frameon=False, prop={'size': 15.})
    plt.title(title)
    
    g.figure.savefig('sim: ' + title + '.png', dpi=150)

### core-transfer (best)

In [ ]:
data = pd.DataFrame()
for experiment_name in ["SIM, core_transfer (best), se2d_fullgaussian2d, 0-0-3 -> 0-0-3",
                         "SIM, core_transfer (best), se2d_pointpooled, 0-0-3 -> 0-0-3",
                         "SIM, core_transfer (best), se2d_spatialxfeaturelinear, 0-0-3 -> 0-0-3"]:
    data_ = pd.DataFrame((TrainedModelTransfer * Dataset * Model * Trainer * Seed * Transfer.proj() * ExperimentsTransfer.Restrictions & 'experiment_name="{}"'.format(experiment_name)).fetch())
    transfer = get_transfer(old_experiment_name='SIM, Direct, {}, 0-0-3'.format(experiment_name.split(', ')[2]))
    data_ = pd.merge(data_, transfer, how='inner', on=['transfer_hash', 'transfer_fn'])
    data = pd.concat([data, data_])
data['readout'] = [row.model_fn.split('.')[-1] for loc, row in data.iterrows()]
data = pd.concat([data, data['dataset_config'].apply(pd.Series)], axis = 1).drop('dataset_config', axis = 1)
base_line = baseline(data=data, tier='validation')

In [ ]:
title = 'Transfer: best core'
neuron_n = 1000

hues = np.unique(data.readout.values)
fig, ax = plt.subplots(1, 1, figsize=(10,10))

sns.set_context('notebook', font_scale=1.45)
pal = sns.color_palette('colorblind', n_colors=len(hues))
sns.set_palette(pal)
with sns.axes_style('ticks'):
    g = sns.pointplot('image_n', "score", hue='readout', data=data.loc[data['neuron_n'] == neuron_n], 
                hue_order=hues, ax=ax)
    plt.axhline(base_line, c='k', ls='--', label='baseline')
    g.set_ylim(0.3, 1.0)
    sns.despine(trim=True)
    g.axes.set_ylabel('validation correlation') 
    g.axes.set_xlabel('images (new readout)') 
    plt.legend(loc='lower right', bbox_to_anchor=(0.99, 0.05), ncol=1, frameon=False, prop={'size': 15.})
    plt.title(title)
    
    g.figure.savefig('sim: ' + title + '.png', dpi=150)